In [17]:
%matplotlib inline
from scipy import signal
from scipy.io.wavfile import write,read
import matplotlib.pyplot as plt
import math
import numpy as np
import IPython
from numpy import convolve
from scipy.signal import hamming
from scipy.ndimage.interpolation import shift
from scipy.linalg import toeplitz,inv
from scipy.fftpack import fft,fftshift,ifft,dct

def get_mel_filters(N,num_filts,Fs,freq_down=300,freq_up=2000):
    up_melcoef=1125*np.log(1+freq_up/700)
    down_melcoef=1125*np.log(1+freq_down/700)
    filtbank_mel=np.linspace(down_melcoef,up_melcoef,num_filts+2)
    filtbank_hz=700*(np.exp(filtbank_mel/1125)-1)
    #print (filtbank_hz)
    filtbank_bins=np.floor((N+1)*filtbank_hz/Fs)
    print(filtbank_hz)
    #print (filtbank_bins)
    #Computing filters now
    filters=np.zeros(shape=(num_filts,N))
    for i in range(1,num_filts):
        fp=int(filtbank_bins[i-1])
        sp=int(filtbank_bins[i])
        tp=int(filtbank_bins[i+1])
        filters[i-1]=np.hstack((np.zeros(fp),np.linspace(0,1,sp-fp+1),np.linspace(1,0,tp-sp+1)[1:],np.zeros(N-tp-1))).ravel()
        #plt.plot(filters[i-1])
    return filters

def feat_ext(digit,num_wav,Fs):
    
    # Parameters
    N=512
    num_filts=20
    mel_filters=get_mel_filters(N,num_filts,Fs)
    t_analysis=0.01
    hop_samp=int(Fs*t_analysis)
    num_samp=hop_samp
    
    for samp_under_analysis in range(1,num_wav+1):

        filename="./Processed_Data/"+str(digit)+"/"+str(samp_under_analysis)+".wav"
        print(filename+ " Read")
        read_wav = read(filename)
        inp_wo_emph=np.array(read_wav[1],dtype='float64')
        inp = np.zeros(inp_wo_emph.size)
        a=0.97
        for k in range (inp_wo_emph.size):
            if(k==0):
                inp[k]=inp_wo_emph[k]
            else:
                inp[k]=inp_wo_emph[k] - a*inp_wo_emph[k-1];

        
        #plt.rcParams["figure.figsize"] = (18,10)
        #plt.plot(inp,'blue')
        num_feat=int((inp.size-num_samp)/hop_samp)
        num_coef=num_filts
        mfcc=np.zeros(shape=(num_feat,num_coef))
        st_ind=0
        #print (inp.size)
        for i in range(num_feat):
    
            end_ind=st_ind+num_samp            
            #print ("Frame number: "+str(i)+" st index "+ str(st_ind)+" end index "+ str(end_ind))
            frame=inp[st_ind:end_ind]
            #---------------------------------------------------------
            #Windowed DFT
            zero_arr=np.zeros(N-num_samp)
            zero_pd=np.append(frame,zero_arr)
            hamm_w=np.append(hamming(num_samp),np.zeros((N-num_samp)))
            s_n=zero_pd*hamm_w ## x[n]=s[n]w[n]
            S_k=fft(s_n)
            #P_k=(np.abs(S_k[0:257])*np.abs(S_k[0:257]))/num_samp
            P_k=np.abs(np.real(S_k))
            #---------------------------------------------------------
            #Computing MFCC
            epsilon=1e-8
            energy_fbank=[20*np.log10(np.sum(P_k*mel_filters[i])+epsilon) for i in range(num_filts)]
            ifft_samp=ifft(energy_fbank)
            #---------------------------------------------------------
            # DCT
            #orig_mfcc=dct(energy_fbank)
            orig_mfcc=ifft_samp
            #if(i>=2 and i<(num_feat-2)):
            #    delta_mfcc=(mfcc[i+1]-mfcc[i-1]+2*mfcc[i+2]-2*mfcc[i-2])/10
            #else:
            #    delta_mfcc=np.zeros(13)
            
            #mfcc[i]=np.append(orig_mfcc,delta_mfcc[0:7])
            mfcc[i]=orig_mfcc
            #print (mfcc[i])
            st_ind=st_ind+hop_samp
        filename="./Extracted_Feats/"+str(digit)+"/"+str(samp_under_analysis)
        print(filename+" Saved")
        np.save(filename,mfcc)
  
feat_ext(0,64,8000)
feat_ext(1,64,8000)
feat_ext(2,64,8000)
feat_ext(3,64,8000)
feat_ext(4,64,8000)
feat_ext(5,64,8000)
feat_ext(6,64,8000)
feat_ext(7,64,8000)
feat_ext(8,64,8000)
feat_ext(9,64,8000)
            

[  300.           348.4340851    399.21403079   452.4534567    508.27148549
   566.79300944   628.14896986   692.47665008   759.91998265   830.62987132
   904.76452876   982.48983051  1063.97968613  1149.41642836  1238.99122103
  1332.90448683  1431.36635574  1534.59713518  1642.82780298  1756.30052416
  1875.26919277  2000.        ]
./Processed_Data/0/1.wav Read
./Extracted_Feats/0/1 Saved
./Processed_Data/0/2.wav Read
./Extracted_Feats/0/2 Saved
./Processed_Data/0/3.wav Read
./Extracted_Feats/0/3 Saved
./Processed_Data/0/4.wav Read
./Extracted_Feats/0/4 Saved
./Processed_Data/0/5.wav Read
./Extracted_Feats/0/5 Saved
./Processed_Data/0/6.wav Read
./Extracted_Feats/0/6 Saved
./Processed_Data/0/7.wav Read
./Extracted_Feats/0/7 Saved
./Processed_Data/0/8.wav Read
./Extracted_Feats/0/8 Saved
./Processed_Data/0/9.wav Read
./Extracted_Feats/0/9 Saved
./Processed_Data/0/10.wav Read


/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:94: ComplexWarning: Casting complex values to real discards the imaginary part


./Extracted_Feats/0/10 Saved
./Processed_Data/0/11.wav Read
./Extracted_Feats/0/11 Saved
./Processed_Data/0/12.wav Read
./Extracted_Feats/0/12 Saved
./Processed_Data/0/13.wav Read
./Extracted_Feats/0/13 Saved
./Processed_Data/0/14.wav Read
./Extracted_Feats/0/14 Saved
./Processed_Data/0/15.wav Read
./Extracted_Feats/0/15 Saved
./Processed_Data/0/16.wav Read
./Extracted_Feats/0/16 Saved
./Processed_Data/0/17.wav Read
./Extracted_Feats/0/17 Saved
./Processed_Data/0/18.wav Read
./Extracted_Feats/0/18 Saved
./Processed_Data/0/19.wav Read
./Extracted_Feats/0/19 Saved
./Processed_Data/0/20.wav Read
./Extracted_Feats/0/20 Saved
./Processed_Data/0/21.wav Read
./Extracted_Feats/0/21 Saved
./Processed_Data/0/22.wav Read
./Extracted_Feats/0/22 Saved
./Processed_Data/0/23.wav Read
./Extracted_Feats/0/23 Saved
./Processed_Data/0/24.wav Read
./Extracted_Feats/0/24 Saved
./Processed_Data/0/25.wav Read
./Extracted_Feats/0/25 Saved
./Processed_Data/0/26.wav Read
./Extracted_Feats/0/26 Saved
./Processed

./Extracted_Feats/2/10 Saved
./Processed_Data/2/11.wav Read
./Extracted_Feats/2/11 Saved
./Processed_Data/2/12.wav Read
./Extracted_Feats/2/12 Saved
./Processed_Data/2/13.wav Read
./Extracted_Feats/2/13 Saved
./Processed_Data/2/14.wav Read
./Extracted_Feats/2/14 Saved
./Processed_Data/2/15.wav Read
./Extracted_Feats/2/15 Saved
./Processed_Data/2/16.wav Read
./Extracted_Feats/2/16 Saved
./Processed_Data/2/17.wav Read
./Extracted_Feats/2/17 Saved
./Processed_Data/2/18.wav Read
./Extracted_Feats/2/18 Saved
./Processed_Data/2/19.wav Read
./Extracted_Feats/2/19 Saved
./Processed_Data/2/20.wav Read
./Extracted_Feats/2/20 Saved
./Processed_Data/2/21.wav Read
./Extracted_Feats/2/21 Saved
./Processed_Data/2/22.wav Read
./Extracted_Feats/2/22 Saved
./Processed_Data/2/23.wav Read
./Extracted_Feats/2/23 Saved
./Processed_Data/2/24.wav Read
./Extracted_Feats/2/24 Saved
./Processed_Data/2/25.wav Read
./Extracted_Feats/2/25 Saved
./Processed_Data/2/26.wav Read
./Extracted_Feats/2/26 Saved
./Processed

./Extracted_Feats/4/16 Saved
./Processed_Data/4/17.wav Read
./Extracted_Feats/4/17 Saved
./Processed_Data/4/18.wav Read
./Extracted_Feats/4/18 Saved
./Processed_Data/4/19.wav Read
./Extracted_Feats/4/19 Saved
./Processed_Data/4/20.wav Read
./Extracted_Feats/4/20 Saved
./Processed_Data/4/21.wav Read
./Extracted_Feats/4/21 Saved
./Processed_Data/4/22.wav Read
./Extracted_Feats/4/22 Saved
./Processed_Data/4/23.wav Read
./Extracted_Feats/4/23 Saved
./Processed_Data/4/24.wav Read
./Extracted_Feats/4/24 Saved
./Processed_Data/4/25.wav Read
./Extracted_Feats/4/25 Saved
./Processed_Data/4/26.wav Read
./Extracted_Feats/4/26 Saved
./Processed_Data/4/27.wav Read
./Extracted_Feats/4/27 Saved
./Processed_Data/4/28.wav Read
./Extracted_Feats/4/28 Saved
./Processed_Data/4/29.wav Read
./Extracted_Feats/4/29 Saved
./Processed_Data/4/30.wav Read
./Extracted_Feats/4/30 Saved
./Processed_Data/4/31.wav Read
./Extracted_Feats/4/31 Saved
./Processed_Data/4/32.wav Read
./Extracted_Feats/4/32 Saved
./Processed

./Extracted_Feats/6/18 Saved
./Processed_Data/6/19.wav Read
./Extracted_Feats/6/19 Saved
./Processed_Data/6/20.wav Read
./Extracted_Feats/6/20 Saved
./Processed_Data/6/21.wav Read
./Extracted_Feats/6/21 Saved
./Processed_Data/6/22.wav Read
./Extracted_Feats/6/22 Saved
./Processed_Data/6/23.wav Read
./Extracted_Feats/6/23 Saved
./Processed_Data/6/24.wav Read
./Extracted_Feats/6/24 Saved
./Processed_Data/6/25.wav Read
./Extracted_Feats/6/25 Saved
./Processed_Data/6/26.wav Read
./Extracted_Feats/6/26 Saved
./Processed_Data/6/27.wav Read
./Extracted_Feats/6/27 Saved
./Processed_Data/6/28.wav Read
./Extracted_Feats/6/28 Saved
./Processed_Data/6/29.wav Read
./Extracted_Feats/6/29 Saved
./Processed_Data/6/30.wav Read
./Extracted_Feats/6/30 Saved
./Processed_Data/6/31.wav Read
./Extracted_Feats/6/31 Saved
./Processed_Data/6/32.wav Read
./Extracted_Feats/6/32 Saved
./Processed_Data/6/33.wav Read
./Extracted_Feats/6/33 Saved
./Processed_Data/6/34.wav Read
./Extracted_Feats/6/34 Saved
./Processed

./Extracted_Feats/8/19 Saved
./Processed_Data/8/20.wav Read
./Extracted_Feats/8/20 Saved
./Processed_Data/8/21.wav Read
./Extracted_Feats/8/21 Saved
./Processed_Data/8/22.wav Read
./Extracted_Feats/8/22 Saved
./Processed_Data/8/23.wav Read
./Extracted_Feats/8/23 Saved
./Processed_Data/8/24.wav Read
./Extracted_Feats/8/24 Saved
./Processed_Data/8/25.wav Read
./Extracted_Feats/8/25 Saved
./Processed_Data/8/26.wav Read
./Extracted_Feats/8/26 Saved
./Processed_Data/8/27.wav Read
./Extracted_Feats/8/27 Saved
./Processed_Data/8/28.wav Read
./Extracted_Feats/8/28 Saved
./Processed_Data/8/29.wav Read
./Extracted_Feats/8/29 Saved
./Processed_Data/8/30.wav Read
./Extracted_Feats/8/30 Saved
./Processed_Data/8/31.wav Read
./Extracted_Feats/8/31 Saved
./Processed_Data/8/32.wav Read
./Extracted_Feats/8/32 Saved
./Processed_Data/8/33.wav Read
./Extracted_Feats/8/33 Saved
./Processed_Data/8/34.wav Read
./Extracted_Feats/8/34 Saved
./Processed_Data/8/35.wav Read
./Extracted_Feats/8/35 Saved
./Processed

# Now Bag of Frames

In [18]:
def validation(st_index,end_index):
    test=[]
    for digits in range(10):
        digit=[]
        for i in range(st_index,end_index):
            filename="./Extracted_Feats/"+str(digits)+"/"+str(i)+".npy"
            digit.append(np.load(filename))
        test.append(digit)
    #print("Test 0,0")
    #print(test[0][3].shape[0])
    #print ("Test")
    #print(test[9][1][0].shape)
    train_samp=[i for i in range(1,65) if i not in range(st_index,end_index)]
    train=[]
    for digits in range(10):
        digit=[]
        for i in train_samp:
            filename="./Extracted_Feats/"+str(digits)+"/"+str(i)+".npy"
            digit.append(np.load(filename))
        train.append(digit)
    #print("Train 0,0")
    #print(train[0][0].shape)
    #print ("Train")
    #print(train[9][1][10].shape)
    for digit_under_analysis in range(0,10):
        print("Testing for "+str(digit_under_analysis))
        merit=np.zeros(shape=(4,10))
        for num_samp in range(4):
            for testing_digit in range(10):
                dist=0
                count=1
                for num_tr_samp in range(len(train[testing_digit])):
                    #test[digit_under_analysis][num_samp], train[testing_digit][num_tr_samp]
                    for num_frame in range(test[digit_under_analysis][num_samp].shape[0]):
                        for num_tr_frame in range(train[testing_digit][num_tr_samp].shape[0]):
                            vec_a=test[digit_under_analysis][num_samp][num_frame]
                            vec_b=train[testing_digit][num_tr_samp][num_tr_frame]
                            #print(vec_a.shape)
                            #print(vec_b.shape)
                            #epsilon=1e-8
                            samp_dist=np.sum(np.square(vec_a-vec_b))
                            #print(samp_dist)
                            dist=dist+samp_dist
                            count=count+1
                merit[num_samp][testing_digit]=dist/count
        print(merit)
        for i in range(4):
            print("Detected Digit for sample: "+ str(i)+ " is :"+ str(np.argmin(merit[i])) )
                            
            
    
validation(41,45)

Testing for 0
[[ 432.09524142  512.96872954  537.68478547  411.1018083   457.09520863
   492.24816127  475.16277888  482.87837124  465.25670472  498.98729978]
 [ 431.36249007  488.64829246  615.04750502  450.11313831  385.78036072
   439.55933524  556.26242706  448.05041843  492.52186209  482.48911286]
 [ 410.01757192  484.79246642  536.8226207   399.69601893  416.01608351
   456.45289775  475.28621083  451.78771246  450.66756172  472.86972238]
 [ 427.51395233  478.18286028  633.11401467  457.49150618  361.79650142
   421.13264236  575.27661291  434.59757546  496.46019554  474.24358011]]
Detected Digit for sample: 0 is :3
Detected Digit for sample: 1 is :4
Detected Digit for sample: 2 is :3
Detected Digit for sample: 3 is :4
Testing for 1
[[ 437.36102069  503.02010119  564.39126003  435.68190354  429.92417067
   474.72924539  507.06441109  475.72655738  485.32914658  495.66330917]
 [ 452.77914108  522.15194328  567.8087268   444.07924     457.19337462
   497.69872144  509.2622763   495